In [1]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 30.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=8da6aac4b90746c987c27926e241c4d52fee68806d3ddf1e8c918c2176fc6f61
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 29.8 MB/s 
     |████████████████████████████████| 4.0 MB 26.5 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 82.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [28]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [29]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [30]:
df['text'] = df['text'].astype('str') 

In [8]:
#df_sample = df.sample(n=50000, random_state=2)

In [9]:
#df_sample

In [10]:
docs = list(df.loc[:, "text"].values)

In [11]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [12]:
178255 * 6

1069530

In [13]:
docs = docs[891275:1069530]

In [14]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [3:05:00<00:00, 16.06it/s]


In [31]:
df = df[891275:1069530]

In [32]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [33]:
df

,text,Label,Negative,Neutral,Positive
891275,Turkey says Ukraine-Russia talks unlikely on W...,1,0.216880,0.776355,0.006765
891276,"EUR/USD, EUR/GBP and AUD/USD sidelined as trad...",1,0.088363,0.902982,0.008655
891277,Russia invades Ukraine the first war I see on ...,0,0.933388,0.063154,0.003458
891278,@TelegraphWorld\n\nStill not enough. \n\nThe w...,0,0.926924,0.069928,0.003149
891279,#UkraineRussia #UkraineWar \nhttps://t.co/tzQ8...,1,0.282529,0.710308,0.007164
...,...,...,...,...,...
1069525,"#Kyiv says 352 civilians have been killed, inc...",0,0.961660,0.035455,0.002885
1069526,@_natrimorrigna Obviously with adding the Balt...,1,0.456645,0.536111,0.007244
1069527,@Mignear @hnjc99 @Inc02 @KyivIndependent Who c...,0,0.968090,0.029024,0.002887
1069528,If you are an international anything with the ...,0,0.960788,0.036053,0.003159


In [34]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p6.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p6.csv 'drive/My Drive/Colab Notebooks'

Mounted at drive
